Premise: train a neural net to add n-bit integers

input data in the form:
X = [[1,0,...,1,0],
     [1,0,...,0,1]]
X.shape = (2,n)
outpud data:
Y = X[0] + x[1] in binary as a list with size (n+1)




In [64]:
from fastai import *
from fastai.data.all import *
import torch

In [239]:
# data generation

def binary(x, bits):
    """takes a tensor of ints and returns a tensor of shape = (*x.shape, bits)
    that represents x in binary bits
    e.g.:
    x = [3,4]
    binary(x,bits=3) --> 
        [[0, 1, 1],  # 3 in binary
         [1, 0, 0]], # 4 in binary"""
    mask = 2**torch.arange(bits-1, -1, -1).to(x.device, x.dtype)
    return x.unsqueeze(-1).bitwise_and(mask).ne(0).byte()

def generate_data(samples, bits = 4):
    """generates `samples` number of samples of 
    2 binary numbers as X and their sum as Y
    X.shape = (samples, 2, bits)
    Y.shape = (samples, bits+1)"""
    X = torch.randint(0,2**bits, (samples,2))
    Y = X.sum(1)
    return (binary(X,bits), binary(Y,bits+1))






In [319]:
def generate_data(samples, bits = 4):
    return torch.randint(0,2, (samples, 2, bits))

In [327]:
def vec_bin_array(arr, m):
    """
    Arguments: 
    arr: Numpy array of positive integers
    m: Number of bits of each integer to retain

    Returns a copy of arr with every element replaced with a bit vector.
    Bits encoded as int8's.
    """
    to_str_func = np.vectorize(lambda x: np.binary_repr(x).zfill(m))
    strs = to_str_func(arr)
    ret = np.zeros(list(arr.shape) + [m], dtype=np.int8)
    for bit_ix in range(0, m):
        fetch_bit_func = np.vectorize(lambda x: x[bit_ix] == '1')
        ret[...,bit_ix] = fetch_bit_func(strs).astype("int8")

    return ret 

def get_y(x):
    x = x.dot(1 << np.arange(x.shape[-1] - 1, -1, -1)).sum()
    return x


db = DataBlock(
    get_items = generate_data,
    splitter = RandomSplitter(),
    get_y = get_y,
)
ds = db.datasets(5)
for sample in ds.train:
    print(sample)

(array([[0, 0, 1, 1],
       [1, 0, 0, 1]], dtype=int64), 12)
(array([[0, 0, 0, 1],
       [0, 1, 1, 0]], dtype=int64), 7)
(array([[0, 0, 1, 0],
       [0, 0, 0, 0]], dtype=int64), 2)
(array([[0, 0, 0, 0],
       [0, 0, 1, 0]], dtype=int64), 2)


In [330]:
class RandDL(DataLoader):
    bits = 4

    def create_item(self, s):
        if random.random() > 0.99: stop()
        return torch.randint(0,2, (2, self.bits))


dl = RandDL(bs=4, drop_last=True)
        

AttributeError: train